# Set A: Qwen3-VL Tool Calling 능력 진단

파인튜닝 전 베이스 모델의 **개별 능력**을 독립적으로 측정한다.  
테스트당 하나의 능력만 측정하며, 프롬프트 지시 수준을 동일하게 유지한다.


| ID | 테스트 | 측정 능력 |
|----|--------|----------|
| A1 | 기본 tool call | 포맷 생성 |
| A2a | catch 호출 | 단일 도구 정확도 |
| A2b | notepad 호출 | 단일 도구 정확도 |
| A2c | declare_found 호출 | 단일 도구 정확도 |
| A3 | 다중 action 구성 | 복합 구조 |
| A4 | 병렬 도구 호출 | 여러 도구 동시 |
| A5 | 좌표 계산 | 수치 추론 |
| A6 | 조건 추론 | 제약 판단 |
| A7 | 이미지→tool call | VLM 연결 |


In [ ]:
# ── Cell 1: 환경 설정 + 경로 ──────────────────────────────────
import os, sys
from dotenv import load_dotenv

load_dotenv()
sys.path.insert(0, os.path.abspath(".."))

print("환경 설정 및 경로 추가 완료")

In [ ]:
# ── Cell 2: LLM 연결 ─────────────────────────────────────────
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

llm_configs = {
    "runpod": {
        "base_url": "https://api.runpod.ai/v2/8iiebdj6zt0fbd/openai/v1",
        "model": "qwen/qwen3-vl-4b-thinking-fp8",
        "api_key": os.getenv("RUNPOD_API_KEY"),
    },
    "local": {
        "base_url": "http://192.168.50.32:8000/v1",
        "model": "Qwen3-VL-2B-Thinking-FP8",
        "api_key": "EMPTY",
    },
}

# ── 사용할 설정 선택 ─────────────────────────────────────────
target = "local"
cfg = llm_configs[target]

VLLM_BASE_URL = cfg["base_url"]
MODEL_NAME = cfg["model"]

print(f"현재 활성화된 설정: [{target}]")
print(f"Base URL: {VLLM_BASE_URL}")
print(f"Model Name: {MODEL_NAME}")

llm = ChatOpenAI(
    base_url=VLLM_BASE_URL,
    model=MODEL_NAME,
    temperature=0,
    api_key=cfg["api_key"],
    max_tokens=10000,
)

response = llm.invoke([HumanMessage(content="연결 테스트. 한 문장으로 답해.")])
print(f"\n[연결 확인] {response.content[:200]}")

In [ ]:
# ── Cell 3: 도구 정의 (catch 추가) + 테스트 하네스 ─────────────
import json as _json
import time as _time
import base64 as _base64
from typing import Literal

from langchain_core.tools import tool
from langchain_core.messages import (
    HumanMessage, AIMessage, ToolMessage, SystemMessage,
)
from langchain_core.utils.function_calling import convert_to_openai_tool
from pydantic import BaseModel, Field

# ── 도구 스키마 ────────────────────────────────────────────

class Action(BaseModel):
    direction: Literal["UP", "DOWN", "LEFT", "RIGHT"] = Field(
        description="이동 방향"
    )
    steps: int = Field(description="이동 칸수 (1~3)", ge=1, le=3)

class MoveArgs(BaseModel):
    actions: list[Action] = Field(description="최대 4개 행동을 순서대로 실행")

@tool(args_schema=MoveArgs)
def move(actions: list[Action]) -> str:
    """플레이어를 이동시킨다."""
    return _json.dumps({"moved": True})

class CatchArgs(BaseModel):
    direction: Literal["UP", "DOWN", "LEFT", "RIGHT"] = Field(
        description="포획할 동물이 있는 방향"
    )

@tool(args_schema=CatchArgs)
def catch_animal(direction: str) -> str:
    """인접 타일(상하좌우)의 동물을 포획한다."""
    return _json.dumps({"success": True, "animal": {"emoji": "\ud83d\udc12"}})

class NotepadArgs(BaseModel):
    content: str = Field(
        description="메모장 전체를 덮어쓴다. 유지할 내용도 포함해서 작성해야 한다. 최대 2000자."
    )

@tool(args_schema=NotepadArgs)
def update_notepad(content: str) -> str:
    """메모장 전체를 덮어쓴다."""
    return _json.dumps({"status": "updated"})

class DeclareFoundArgs(BaseModel):
    target: str = Field(description="찾은 타겟 이름")

@tool(args_schema=DeclareFoundArgs)
def declare_found(target: str) -> str:
    """특정 타겟을 찾아서 도달했음을 선언한다."""
    return _json.dumps({"declared": True})

class DeclareDoneArgs(BaseModel):
    reason: str = Field(default="", description="미션 완료 사유")

@tool(args_schema=DeclareDoneArgs)
def declare_done(reason: str = "") -> str:
    """전체 미션이 완료되었음을 선언한다."""
    return _json.dumps({"done": True})

ALL_TOOLS = [move, catch_animal, update_notepad, declare_found, declare_done]

SYSTEM_MSG = (
    "너는 50x50 격자 맵 위의 사파리 에이전트야. "
    "반드시 제공된 도구를 사용해서 행동해. 텍스트로 답하지 말고 도구를 호출해. "
    "좌표계: x축은 RIGHT(+)/LEFT(-), y축은 DOWN(+)/UP(-)."
)

# ── args list→dict 자동 보정 fallback ────────────────────────

def _to_openai_messages(messages):
    """langchain 메시지 리스트를 OpenAI API 형식으로 변환."""
    result = []
    for m in messages:
        if isinstance(m, SystemMessage):
            result.append({"role": "system", "content": m.content})
        elif isinstance(m, HumanMessage):
            result.append({"role": "user", "content": m.content})
        elif isinstance(m, AIMessage):
            msg = {"role": "assistant", "content": m.content or ""}
            if m.tool_calls:
                msg["tool_calls"] = [
                    {
                        "id": tc["id"],
                        "type": "function",
                        "function": {
                            "name": tc["name"],
                            "arguments": _json.dumps(tc["args"]),
                        },
                    }
                    for tc in m.tool_calls
                ]
            result.append(msg)
        elif isinstance(m, ToolMessage):
            result.append({
                "role": "tool",
                "content": m.content,
                "tool_call_id": m.tool_call_id,
            })
    return result


def _fix_tool_call_args(raw_tool_calls):
    """
    raw OpenAI tool_calls에서 args가 list인 경우 {"actions": list}로 래핑.
    모델이 간헐적으로 wrapper dict를 생략하는 비결정적 포맷 오류를 보정한다.
    """
    fixed = []
    for tc in raw_tool_calls:
        args = _json.loads(tc.function.arguments)
        if isinstance(args, list):
            args = {"actions": args}
            print(f"  [보정] {tc.function.name}: args list → {{\"actions\": [...]}} 래핑")
        fixed.append({
            "name": tc.function.name,
            "args": args,
            "id": tc.id,
            "type": "tool_call",
        })
    return fixed


class _FallbackResponse:
    """pydantic 검증 실패 시 raw OpenAI 응답을 langchain 호환 형태로 감싸는 래퍼."""
    def __init__(self, content, tool_calls, response_metadata):
        self.content = content
        self.tool_calls = tool_calls
        self.response_metadata = response_metadata


def _invoke_with_tool_fallback(llm_instance, tools, messages):
    """
    bind_tools 기반 호출 후, args가 list로 반환되어 pydantic 검증 실패하면
    raw OpenAI client로 재호출하여 args를 dict로 자동 래핑한다.
    """
    llm_with_tools = llm_instance.bind_tools(tools)
    try:
        return llm_with_tools.invoke(messages)
    except Exception as e:
        err_str = str(e)
        if "input_type=list" not in err_str or "dict_type" not in err_str:
            raise

    # ── Fallback: raw OpenAI client 재호출 ──
    print("  [보정] pydantic validation error 감지 → raw client fallback")
    openai_tools = [convert_to_openai_tool(t) for t in tools]
    openai_msgs = _to_openai_messages(messages)

    raw_resp = llm_instance.client.create(
        model=llm_instance.model_name,
        messages=openai_msgs,
        tools=openai_tools,
        temperature=0,
        max_tokens=llm_instance.max_tokens,
    )

    choice = raw_resp.choices[0]
    msg = choice.message

    tool_calls = _fix_tool_call_args(msg.tool_calls) if msg.tool_calls else []
    usage = raw_resp.usage

    return _FallbackResponse(
        content=msg.content or "",
        tool_calls=tool_calls,
        response_metadata={
            "token_usage": {
                "prompt_tokens": usage.prompt_tokens if usage else 0,
                "completion_tokens": usage.completion_tokens if usage else 0,
                "total_tokens": usage.total_tokens if usage else 0,
            }
        },
    )

# ── 테스트 하네스 ────────────────────────────────────────────

def _print_response_text(resp, passed):
    """응답 텍스트 출력. 실패 시 전체 텍스트, 성공 시 200자 요약."""
    if not resp.content:
        return
    content = resp.content
    if "</think>" in content:
        content = content.split("</think>")[-1].strip()
    if not content:
        return
    if passed:
        print(f"  응답 텍스트: {content[:200]}")
    else:
        print(f"  응답 텍스트 (전체 — tool call 실패 시 모델 출력):")
        for line in content.splitlines():
            print(f"    {line}")

def run_test(
    test_id: str,
    title: str,
    messages: list,
    tools: list = None,
    validator=None,
    llm_override=None,
) -> dict:
    """단일 테스트 실행 및 결과 출력. llm_override로 별도 LLM 인스턴스 사용 가능."""
    if tools is None:
        tools = ALL_TOOLS
    llm_instance = llm_override or llm

    print(f"\n{'='*60}")
    print(f"  {test_id}: {title}")
    print(f"{'='*60}")

    start = _time.time()
    try:
        resp = _invoke_with_tool_fallback(llm_instance, tools, messages)
        elapsed = _time.time() - start
    except Exception as e:
        elapsed = _time.time() - start
        print(f"\n  FAIL - 호출 실패: {e}")
        return {
            "id": test_id, "title": title, "passed": False,
            "reason": str(e), "tool_calls": [], "tokens": {}, "elapsed": elapsed,
        }

    usage = resp.response_metadata.get("token_usage", {})
    tokens = {
        "prompt": usage.get("prompt_tokens", "?"),
        "completion": usage.get("completion_tokens", "?"),
        "total": usage.get("total_tokens", "?"),
    }

    tool_calls = resp.tool_calls or []

    if validator:
        passed, reason = validator(resp, tool_calls)
    else:
        passed = len(tool_calls) > 0
        reason = "tool call 존재" if passed else "tool call 없음"

    status = "PASS" if passed else "FAIL"

    print(f"\n  상태: {status}")
    print(f"  사유: {reason}")
    print(f"  소요시간: {elapsed:.1f}s")
    print(f"  토큰: prompt={tokens['prompt']}, completion={tokens['completion']}, total={tokens['total']}")
    print(f"  Tool calls ({len(tool_calls)}개):")
    for i, tc in enumerate(tool_calls):
        print(f"    [{i}] {tc['name']}({_json.dumps(tc['args'], ensure_ascii=False)})")

    _print_response_text(resp, passed)

    return {
        "id": test_id, "title": title, "passed": passed,
        "reason": reason, "tool_calls": tool_calls,
        "tokens": tokens, "elapsed": elapsed,
    }

results = []

print("테스트 하네스 및 도구 정의 완료")
print(f"사용 가능 도구: {[t.name for t in ALL_TOOLS]}")

## 테스트 아키텍처

**설계 원칙:**
- 테스트당 **하나의 능력**만 측정
- 프롬프트 지시 수준 동일 (명시적 지시)
- 좌표계 통일: `UP=y-1, DOWN=y+1, RIGHT=x+1, LEFT=x-1` (실제 게임과 동일)
- 5개 도구 전체 커버: `move`, `catch_animal`, `update_notepad`, `declare_found`, `declare_done`

**실패 해석:** 각 테스트가 단일 능력만 측정하므로, 실패 시 해당 능력이 약점임을 직접 판단할 수 있다.


In [ ]:
# ── A1: 기본 tool call ────────────────────────────────────────
# 측정 능력: 포맷 생성
# "오른쪽 2칸 이동해" → move(RIGHT, 2)

def validate_a1(resp, tool_calls):
    if not tool_calls:
        return False, "tool call 없음"
    move_calls = [tc for tc in tool_calls if tc["name"] == "move"]
    if not move_calls:
        return False, "move 호출 없음"
    actions = move_calls[0]["args"].get("actions", [])
    if len(actions) == 1 and actions[0]["direction"] == "RIGHT" and actions[0]["steps"] == 2:
        return True, "move(RIGHT, 2) 정확 호출"
    return False, f"예상과 다른 호출: {actions}"

result = run_test(
    test_id="A1",
    title="기본 tool call",
    messages=[
        SystemMessage(content=SYSTEM_MSG),
        HumanMessage(content="오른쪽으로 2칸 이동해."),
    ],
    tools=[move],
    validator=validate_a1,
)
results.append(result)

In [ ]:
# ── A2a: catch 호출 ──────────────────────────────────────────
# 측정 능력: 단일 도구 정확도 (catch)
# "오른쪽 동물 포획해" → catch_animal(RIGHT)

def validate_a2a(resp, tool_calls):
    if not tool_calls:
        return False, "tool call 없음"
    catch_calls = [tc for tc in tool_calls if tc["name"] == "catch_animal"]
    if not catch_calls:
        return False, f"catch_animal 호출 없음 (호출된 도구: {[tc['name'] for tc in tool_calls]})"
    direction = catch_calls[0]["args"].get("direction")
    if direction == "RIGHT":
        return True, "catch_animal(RIGHT) 정확 호출"
    return False, f"방향 불일치: {direction} (expected RIGHT)"

result = run_test(
    test_id="A2a",
    title="catch 호출",
    messages=[
        SystemMessage(content=SYSTEM_MSG),
        HumanMessage(content="오른쪽에 동물이 있어. 포획해."),
    ],
    tools=[catch_animal],
    validator=validate_a2a,
)
results.append(result)

In [ ]:
# ── A2b: notepad 호출 ────────────────────────────────────────
# 측정 능력: 단일 도구 정확도 (notepad)
# "메모장에 기록해" → update_notepad(content)

def validate_a2b(resp, tool_calls):
    if not tool_calls:
        return False, "tool call 없음"
    notepad_calls = [tc for tc in tool_calls if tc["name"] == "update_notepad"]
    if not notepad_calls:
        return False, f"update_notepad 호출 없음 (호출된 도구: {[tc['name'] for tc in tool_calls]})"
    content = notepad_calls[0]["args"].get("content", "")
    if content:
        return True, f"update_notepad 호출 + 내용 포함 ({len(content)}자)"
    return False, "content가 비어있음"

result = run_test(
    test_id="A2b",
    title="notepad 호출",
    messages=[
        SystemMessage(content=SYSTEM_MSG),
        HumanMessage(content=(
            "메모장에 다음을 기록해: "
            "[맵] 북쪽에 호랑이 발견 [계획] 3칸 이동 후 포획 시도"
        )),
    ],
    tools=[update_notepad],
    validator=validate_a2b,
)
results.append(result)

In [ ]:
# ── A2c: declare_found 호출 ──────────────────────────────────
# 측정 능력: 단일 도구 정확도 (declare_found)
# "호랑이 발견 선언해" → declare_found("호랑이")

def validate_a2c(resp, tool_calls):
    if not tool_calls:
        return False, "tool call 없음"
    df_calls = [tc for tc in tool_calls if tc["name"] == "declare_found"]
    if not df_calls:
        return False, f"declare_found 호출 없음 (호출된 도구: {[tc['name'] for tc in tool_calls]})"
    target = df_calls[0]["args"].get("target", "")
    if "호랑이" in target or "tiger" in target.lower():
        return True, f"declare_found 호출 + '호랑이' 포함 (target='{target}')"
    return False, f"target에 '호랑이' 없음: '{target}'"

result = run_test(
    test_id="A2c",
    title="declare_found 호출",
    messages=[
        SystemMessage(content=SYSTEM_MSG),
        HumanMessage(content="호랑이를 발견했어. 발견 선언해."),
    ],
    tools=[declare_found],
    validator=validate_a2c,
)
results.append(result)

In [ ]:
# ── A3: 다중 action 구성 ─────────────────────────────────────
# 측정 능력: 복합 구조 구성 (방향/칸수 명시 → 좌표 계산 불필요)
# "RIGHT 3, DOWN 2, RIGHT 1 순서로" → move([{RIGHT,3},{DOWN,2},{RIGHT,1}])

def validate_a3(resp, tool_calls):
    if not tool_calls:
        return False, "tool call 없음"
    move_calls = [tc for tc in tool_calls if tc["name"] == "move"]
    if not move_calls:
        return False, "move 호출 없음"
    actions = move_calls[0]["args"].get("actions", [])
    expected = [
        {"direction": "RIGHT", "steps": 3},
        {"direction": "DOWN", "steps": 2},
        {"direction": "RIGHT", "steps": 1},
    ]
    if len(actions) != 3:
        return False, f"action 수 불일치: {len(actions)}개 (expected 3)"
    for i, (a, e) in enumerate(zip(actions, expected)):
        if a["direction"] != e["direction"] or a["steps"] != e["steps"]:
            return False, f"action[{i}] 불일치: {a} (expected {e})"
    return True, "정확히 3개 action 순서대로 호출"

result = run_test(
    test_id="A3",
    title="다중 action 구성",
    messages=[
        SystemMessage(content=SYSTEM_MSG),
        HumanMessage(content=(
            "move 도구를 한 번 호출해서 다음 순서대로 이동해: "
            "RIGHT 3칸, DOWN 2칸, RIGHT 1칸."
        )),
    ],
    tools=[move],
    validator=validate_a3,
)
results.append(result)

In [ ]:
# ── A4: 병렬 도구 호출 ───────────────────────────────────────
# 측정 능력: 여러 도구 동시 호출
# "이동하고 메모장 기록해" → move(...) + update_notepad(...)

def validate_a4(resp, tool_calls):
    if not tool_calls:
        return False, "tool call 없음"
    names = [tc["name"] for tc in tool_calls]
    has_move = "move" in names
    has_notepad = "update_notepad" in names
    if has_move and has_notepad:
        return True, f"move + update_notepad 동시 호출 ({len(tool_calls)}개)"
    missing = []
    if not has_move:
        missing.append("move")
    if not has_notepad:
        missing.append("update_notepad")
    return False, f"누락: {missing}, 호출된 도구: {names}"

result = run_test(
    test_id="A4",
    title="병렬 도구 호출",
    messages=[
        SystemMessage(content=SYSTEM_MSG),
        HumanMessage(content=(
            "오른쪽 1칸 이동하고, 메모장에 '탐색 시작'이라고 기록해. "
            "두 도구를 모두 호출해."
        )),
    ],
    tools=[move, update_notepad],
    validator=validate_a4,
)
results.append(result)

In [ ]:
# ── A5: 좌표 계산 ────────────────────────────────────────────
# 측정 능력: 수치 추론 (action 1개로 충분 → 구조 복잡도 배제)
# "(10,10)→(13,10) 이동" → move([{RIGHT,3}]) (net delta x=+3, y=0)

def validate_a5(resp, tool_calls):
    if not tool_calls:
        return False, "tool call 없음"
    move_calls = [tc for tc in tool_calls if tc["name"] == "move"]
    if not move_calls:
        return False, "move 호출 없음"

    # 모든 move call의 actions를 합산
    dx, dy = 0, 0
    for mc in move_calls:
        for a in mc["args"].get("actions", []):
            d, s = a["direction"], a["steps"]
            if d == "RIGHT": dx += s
            elif d == "LEFT": dx -= s
            elif d == "DOWN": dy += s
            elif d == "UP": dy -= s

    if dx == 3 and dy == 0:
        return True, f"net delta = (+3, 0) 정확"
    return False, f"net delta = ({dx:+d}, {dy:+d}), expected (+3, 0)"

result = run_test(
    test_id="A5",
    title="좌표 계산",
    messages=[
        SystemMessage(content=SYSTEM_MSG),
        HumanMessage(content=(
            "현재 위치 (10,10)에서 (13,10)으로 이동해. "
            "좌표계: x는 RIGHT(+)/LEFT(-), y는 DOWN(+)/UP(-). "
            "move 도구를 호출해."
        )),
    ],
    tools=[move],
    validator=validate_a5,
)
results.append(result)

In [ ]:
# ── A6: 조건 추론 ────────────────────────────────────────────
# 측정 능력: 제약 판단 (좌표 계산 불필요 → 조건만 테스트)
# "RIGHT 막힘, DOWN으로 우회해" → move에서 RIGHT 미사용 + DOWN 사용

def validate_a6(resp, tool_calls):
    if not tool_calls:
        return False, "tool call 없음"
    move_calls = [tc for tc in tool_calls if tc["name"] == "move"]
    if not move_calls:
        return False, "move 호출 없음"

    has_right = False
    has_down = False
    for mc in move_calls:
        for a in mc["args"].get("actions", []):
            if a["direction"] == "RIGHT":
                has_right = True
            if a["direction"] == "DOWN":
                has_down = True

    if has_right:
        return False, "RIGHT 사용됨 (막힌 방향)"
    if not has_down:
        return False, "DOWN 미사용 (우회 방향)"
    return True, "RIGHT 미사용 + DOWN 사용 (조건 추론 성공)"

result = run_test(
    test_id="A6",
    title="조건 추론",
    messages=[
        SystemMessage(content=SYSTEM_MSG),
        HumanMessage(content=(
            "오른쪽(RIGHT)은 장애물로 막혀있어. "
            "아래쪽(DOWN)으로 우회해서 이동해. move 도구를 호출해."
        )),
    ],
    tools=[move],
    validator=validate_a6,
)
results.append(result)

In [ ]:
# ── A7: 이미지→tool call ─────────────────────────────────────
# 측정 능력: VLM 연결 (이미지 분석 → tool call)
# 스크린샷 + "빨간 동물 방향으로 이동" → move 호출 + 방향 합리적
#
# max_tokens 확장: 이미지 분석 시 thinking 토큰 소진으로
# tool call JSON 미출력 방지 (기본 10000 → 16000)

sample_img_path = "images/safari-sample.png"
with open(sample_img_path, "rb") as f:
    _b64 = _base64.b64encode(f.read()).decode()
_image_url = f"data:image/png;base64,{_b64}"
print(f"이미지 로드: {sample_img_path} ({len(_b64)} bytes base64)")

# A7 전용 LLM 인스턴스 (max_tokens 확장)
llm_a7 = ChatOpenAI(
    base_url=VLLM_BASE_URL,
    model=MODEL_NAME,
    temperature=0,
    api_key=cfg["api_key"],
    max_tokens=16000,
)
print(f"A7 전용 LLM: max_tokens=16000")

def validate_a7(resp, tool_calls):
    if not tool_calls:
        return False, "tool call 없음"
    move_calls = [tc for tc in tool_calls if tc["name"] == "move"]
    if not move_calls:
        return False, f"move 호출 없음 (호출된 도구: {[tc['name'] for tc in tool_calls]})"
    actions = move_calls[0]["args"].get("actions", [])
    if actions:
        return True, f"이미지 분석 → move 호출 (방향: {actions[0]['direction']}, actions={len(actions)}개)"
    return False, "move 호출되었으나 actions 비어있음"

result = run_test(
    test_id="A7",
    title="이미지→tool call",
    messages=[
        SystemMessage(content=SYSTEM_MSG),
        HumanMessage(content=[
            {"type": "text", "text": (
                "이 이미지는 사파리 격자 맵이야. "
                "빨간색 배경의 동물을 찾아서 그쪽으로 이동해. move 도구를 호출해."
            )},
            {"type": "image_url", "image_url": {"url": _image_url}},
        ]),
    ],
    tools=[move],
    validator=validate_a7,
    llm_override=llm_a7,
)
results.append(result)

In [ ]:
# ── 진단 결과 요약표 ─────────────────────────────────────────

print(f"\n{'='*70}")
print(f"  Set A: 능력 진단 결과")
print(f"{'='*70}")

header = f"  {'ID':<6} {'테스트':<22} {'측정 능력':<16} {'결과':<6} {'토큰':>8} {'시간':>7}"
print(f"\n{header}")
print(f"  {'-'*6} {'-'*22} {'-'*16} {'-'*6} {'-'*8} {'-'*7}")

ability_map = {
    "A1": "포맷 생성",
    "A2a": "단일 도구(catch)",
    "A2b": "단일 도구(notepad)",
    "A2c": "단일 도구(declare)",
    "A3": "복합 구조",
    "A4": "병렬 호출",
    "A5": "수치 추론",
    "A6": "제약 판단",
    "A7": "VLM 연결",
}

for r in results:
    tid = r["id"]
    title = r["title"][:20]
    ability = ability_map.get(tid, "")
    status = "PASS" if r["passed"] else "FAIL"
    tok = str(r["tokens"].get("total", "?"))
    elapsed = f"{r['elapsed']:.1f}s"
    print(f"  {tid:<6} {title:<22} {ability:<16} {status:<6} {tok:>8} {elapsed:>7}")

passed = sum(1 for r in results if r["passed"])
total = len(results)

print(f"\n  통과: {passed}/{total}")

# 실패 분석
failed = [r for r in results if not r["passed"]]
if failed:
    print(f"\n  실패 항목:")
    for r in failed:
        print(f"    {r['id']} ({ability_map.get(r['id'], '')}): {r['reason']}")

# Set B 교차 분석용 결과 딕셔너리
diag_results = {r["id"]: r["passed"] for r in results}
print(f"\n  Set B 교차 분석용 결과 딕셔너리:")
print(f"  diag_results = {diag_results}")
print(f"\n  위 딕셔너리를 qwen3-vl-scenario.ipynb의 교차 분석 셀에 복사하세요.")